In [1]:
import pandas as pd
import os, re
import glob

os.chdir(r'c:\doit\practice')

files = glob.glob('card*')           # glob(외장함수) :메타문자 사용

f2 = files[0]                  # files[index]
pd.read_csv(f2, encoding= 'cp949')
pd.read_csv(f2, encoding= 'cp949').dtypes     # parse_dates사용(숫자-> 날짜)


df_list = []
for file in files:
    try:
        raw = pd.read_csv(file, encoding='cp949', parse_dates=[0]) 
        df_list.append(raw)
    except:
        raw = pd.read_csv(file, encoding='utf-8', parse_dates=[0])
        df_list.append(raw)

df = pd.concat(df_list)   # 리스트를 합침

df.reset_index(drop =True, inplace = True)
# 불필요 컬럼삭제 연월일 추가
df = df.drop('등록일자', axis=1)
df.insert(1, 'year', df['사용일자'].dt.year)
df.insert(2, 'month', df['사용일자'].dt.month)
df.insert(3, 'day', df['사용일자'].dt.day)

df['total'] = df['승차총승객수'] + df['하차총승객수']  # 컬럼추가
df.sort_values(by ='total', ascending = False).head(10)  # 내림차순 정렬
df.sort_values(by ='total', ascending = False).tail()


df.groupby('역명').sum().sort_values(by='total', ascending=False)[['total']].head(10)
                   # 가장 많이 사용한 지하철 역
top10_station = df.groupby('역명').sum().sort_values(by='total', ascending=False)[['total']].head(10).index
top10_station_cond = df['역명'].isin(top10_station)

# 승객수 top10
top10 = df[top10_station_cond]
top10
top10.노선명.unique()   #  중복제거

top10.groupby('노선명').sum().sort_values(by ='total', ascending=False)

# 승객수 top5
top5_line = top10.groupby('노선명').sum().sort_values(by='total', ascending=False).head().index
top5_line_cond = df['노선명'].isin(top5_line)

# top10_station top5_lines 조건 필터 :역 True 노선 True
top = df[top10_station_cond & top5_line_cond]
top.sort_values(by = 'total', ascending = False)

# 피벗테이블
month_station=top.pivot_table(index='month', columns='역명', values='total')
month_station


역명,가산디지털단지,강남,고속터미널,구로디지털단지,사당,서울역,선릉,신림,잠실(송파구청),홍대입구
month,,,,,,,,,,
2,62924.892857,128262.821429,46510.428571,87935.321429,47321.142857,41474.410714,73974.678571,100276.642857,106530.785714,94077.392857
3,63741.161290,125027.064516,44173.854839,88651.774194,48378.983871,41016.322581,75230.096774,101544.548387,104659.451613,94844.870968
4,71329.200000,143798.800000,54316.583333,99938.433333,58290.700000,51133.066667,85726.966667,113709.066667,144153.466667,109695.300000
5,72791.516129,151591.322581,61968.387097,104746.419355,63561.935484,58513.483871,90021.096774,118474.645161,140976.548387,125208.387097
6,72629.033333,146197.533333,59985.483333,104675.600000,62524.733333,58191.516667,90955.733333,109221.700000,135699.666667,119819.166667


In [2]:
top.sort_values(by='total', ascending = False).head(10) # 2호선, 4월
top_april = top[top['month'] ==4]     # top 4월만

top_april_day_station = top_april.pivot_table('total', index='day', columns='역명')  # 피벗테이블

df.groupby('month').sum()[['total']].sort_values(by='total', ascending = False) 

,total
month,
5,406148909
6,382106244
4,367720014
3,326397672
2,284212374


In [3]:
# 5월 데이터 추출
may = df[df.month == 5]
may.groupby('day').sum()[['total']].sort_values(by='total', ascending = False) #5월 승객

# 승객수 상위 5개 일자
top5_days = may.groupby('day').sum()[['total']].sort_values(by='total', ascending=False).head().index.values

# 승객수 많은 역
may.groupby('역명').sum()[['total']].sort_values(by='total', ascending = False)

# 승객수 많은 상위 10개 역
top10_stations = may.groupby('역명').sum()[['total']].sort_values(by='total', ascending=False).head(10).index.values
top10_stations

# 피멋테이블(인기 날 역망)
may_table = may.pivot_table(index ='역명', columns='day', values='total')
may_table.loc[top10_stations, top5_days]

day,4,13,27,20,31
역명,,,,,
잠실(송파구청),107634.000000,98433.000000,98906.500000,101506.500000,93341.000000
고속터미널,59351.666667,60265.000000,60415.666667,61099.333333,50759.333333
서울역,36803.400000,38210.400000,39152.200000,38618.800000,34025.000000
홍대입구,55423.000000,56649.666667,54505.333333,56564.333333,49752.000000
강남,189056.000000,193479.000000,192601.000000,193465.000000,177336.000000
사당,73995.000000,77655.000000,76860.500000,76656.000000,73213.000000
신림,137574.000000,139400.000000,137137.000000,138014.000000,65884.500000
선릉,77652.500000,76709.500000,77177.000000,76794.000000,79185.500000
가산디지털단지,71176.000000,69105.500000,68667.500000,68405.000000,71164.500000


In [4]:
# 2월 자료
feb = df[df.month ==2]
feb.groupby(['day', '노선명']).total.max()
feb.groupby(['day', '노선명']).total.min()

# 14일만 추출
feb.groupby(['day', '노선명']).total.max().loc[14]

# total 기준 그룹별 높은 행

get_maximum = lambda x: x.loc[x.total.idxmax()]
feb_grouped = feb.groupby(['day', '노선명']).apply(get_maximum)
feb_grouped

# 14일만 선택
valentine = feb_grouped.loc[14]
valentine
valentine.head(10)

,사용일자,year,month,day,노선명,역명,승차총승객수,하차총승객수,total
노선명,,,,,,,,,
1호선,2022-02-14,2022,2,14,1호선,서울역,38551,37346,75897
2호선,2022-02-14,2022,2,14,2호선,강남,81670,79216,160886
3호선,2022-02-14,2022,2,14,3호선,양재(서초구청),36379,39347,75726
4호선,2022-02-14,2022,2,14,4호선,수유(강북구청),31244,31313,62557
5호선,2022-02-14,2022,2,14,5호선,여의도,28391,28078,56469
6호선,2022-02-14,2022,2,14,6호선,공덕,17308,16927,34235
7호선,2022-02-14,2022,2,14,7호선,가산디지털단지,45973,45137,91110
8호선,2022-02-14,2022,2,14,8호선,문정,22204,23134,45338
9호선,2022-02-14,2022,2,14,9호선,신논현,29672,29219,58891
